In [1]:
"""Auto default of settlement dates based on derived properties and calendars

This notebook shows how you can auto default the settlement date onto a transaction based on a derived instrument property and a calendar

Attributes
----------
transactions
derived properties
calendars
"""

'Auto default of settlement dates based on derived properties and calendars\n\nThis notebook shows how you can auto default the settlement date onto a transaction based on a derived instrument property and a calendar\n\nAttributes\n----------\ntransactions\nderived properties\ncalendars\n'

# 1. Introduction

In this notebook we show how you can auto default the settlement date onto a transaction based on a derived instrument property and a calendar. For the purposes of the demo, we consider a US and UK equity transaction with a trade date of Friday 3 September. 

In [2]:
import lusid
import lusid.models as models
from lusidjam import RefreshingToken

from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.utilities import identify_cash_items
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)

# Import Libraries
from datetime import datetime, timedelta, time
import pytz
import uuid
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import os
import json
import requests
from IPython.core.display import display, HTML

# pandas config
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

display(HTML("<style>.container { width:90% !important; }</style>"))

# pandas config
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)


print("LUSID Environment Initialised")
print(
    "LUSID version : ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
LUSID version :  0.6.7739.0


In [3]:
# Initiate the APIs

transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)
corporate_actions_sources_api = api_factory.build(lusid.api.CorporateActionSourcesApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
calendars_api = api_factory.build(lusid.api.CalendarsApi)

In [4]:
scope = "sd-ibor"

# 2. Load data file

First we load a file of transaction data.

In [5]:
transactions_df = pd.read_csv("data/equity_txn_sd.csv")
portfolio_code = transactions_df["portfolio_code"].unique()[0]
transactions_df["txn_trade_date"] = transactions_df["txn_trade_date"].apply(
    lambda x: pd.to_datetime(x).isoformat()
)
transactions_df

,portfolio_code,portfolio_base_currency,ticker,instrument_type,instrument_id,instrument_currency,name,txn_id,txn_type,txn_trade_date,txn_units,txn_price,txn_consideration
0,GLOBAL_EQ,USD,GB0002162385,equity,D76B447ADA6446878B2BEC9437CF717C,GBP,Aviva,trd_0001,StockIn,2021-09-03T00:00:00,120000,5,600000
1,GLOBAL_EQ,USD,GB0002162385,equity,8B6F3204021C408B8CE5F7DE465A7046,USD,IBM,trd_0002,StockIn,2021-09-03T00:00:00,12000,5,60000


# 3. Load instruments

Next we create two instruments, which we'll use for demonstration. We have one USD and one GBP instrument.

In [6]:
# Load the instrument into LUSID

instrument_identifier_mapping = {"ClientInternal": "instrument_id"}

instrument_mapping_required = {"name": "name"}

responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=transactions_df,
    mapping_required=instrument_mapping_required,
    mapping_optional={},
    file_type="instrument",
    identifier_mapping=instrument_identifier_mapping,
    property_columns=["instrument_currency", "instrument_type"],
    properties_scope=scope,
)

succ, failed, errors = format_instruments_response(responses)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,2,0,0


## 3.1 Create derived property

Next we create a derived property to represent the `SettleDateOffset` which is the days between trade date and settle date for an instruments. In this example, we derive the settle date from the instrument type and instrument currency. We give the USD equities an offset of 3, and the GBP equities an offset of 2.

In [7]:
deriv_formula = f"""
                if((Properties[Instrument/{scope}/instrument_type] eq 'equity') and (Properties[Instrument/{scope}/instrument_currency] eq 'GBP')) 
                then 2 
                else if((Properties[Instrument/{scope}/instrument_type] eq 'equity') and (Properties[Instrument/{scope}/instrument_currency] eq 'USD')) 
                then 3 
                else 1""".replace(
    "\n", ""
)

In [8]:
try:

    properties_api.create_derived_property_definition(
        create_derived_property_definition_request=models.CreateDerivedPropertyDefinitionRequest(
            domain="Instrument",
            scope=scope,
            code="SettleDateOffset",
            display_name="Settle Date offset",
            data_type_id=models.ResourceId(code="number", scope="system"),
            property_description="Settle Date offset",
            derivation_formula=deriv_formula,
        )
    )

except:
    pass

# 4. Create portfolio

In [9]:
try:

    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id GLOBAL_EQ because it already exists in scope sd-ibor.


# 5. Update transactions

In the example below, we use derived properties and calendars to calculate the Settle Date for two transactions booked with a Trade Date of Friday 3 September 2021.

* The IBM stock has an instrument currency of USD and therefore we use the US calendar and a settle date offset of 3. Monday September 6 is a public holiday in the US, and therefore the Settle Date of this transaction is <b>9 September</b>.
* The Aviva stock has an instrument currency of GBP and therefore we use a UK calendar and a settle date offset of 2. Monday September 6 is not a public holiday, and therefore the Settle Date of this transaction is the <b>7 September</b>.


In [10]:
def update_transactions_with_settle_date(
    transactions_df, calendar_scope="CoppClarkHolidayCalendars"
):
    def get_instrument_sd_offset(instrument_id):

        instrument = instruments_api.get_instrument(
            identifier_type="ClientInternal",
            identifier=instrument_id,
            property_keys=[f"Instrument/{scope}/SettleDateOffset"],
        )

        sd_offset = instrument.properties[0].value.metric_value.value

        return int(sd_offset)

    def calc_sd(trade_date, currency, offset, scope="CoppClarkHolidayCalendars"):

        sd = calendars_api.add_business_days_to_date(
            scope=scope,
            add_business_days_to_date_request=models.AddBusinessDaysToDateRequest(
                business_day_offset=offset,
                holiday_codes=[currency],
                start_date=trade_date,
            ),
        )

        return sd.value.isoformat()

    settle_dates = []

    for index, row in transactions_df.iterrows():

        sd_offset = get_instrument_sd_offset(row["instrument_id"])
        sd_currency = row["instrument_currency"]
        trade_date = row["txn_trade_date"][:10]

        settle_date = calc_sd(trade_date, sd_currency, sd_offset)

        settle_dates.append(settle_date)

    transactions_df["settle_date"] = settle_dates

    return transactions_df

In [11]:
transaction_df = update_transactions_with_settle_date(
    transactions_df, calendar_scope="CoppClarkHolidayCalendars"
)

In [12]:
transaction_df[["name", "txn_trade_date", "settle_date"]]

,name,txn_trade_date,settle_date
0,Aviva,2021-09-03T00:00:00,2021-09-07T00:00:00+00:00
1,IBM,2021-09-03T00:00:00,2021-09-09T00:00:00+00:00
